In [2]:
!ls ../generation-data

data.csv	data_large.ndjson	  data.parquet	      generate.py
data.json	data_large.parquet	  data_plain.parquet  __pycache__
data_large.csv	data_large_plain.parquet  generate_large.py


In [3]:
import pandas as pd
import polars as pl

In [4]:
%%time

print("A")

A
CPU times: user 58 μs, sys: 3 μs, total: 61 μs
Wall time: 58.4 μs


In [5]:
import polars as pl

# JSON → Parquet
df_small = pl.read_json("../generation-data/data.json")
df_small.write_parquet(
    "../generation-data/data_plain.parquet",
    use_pyarrow=True,
    compression="snappy"
)

In [6]:
import sys
print(sys.executable)

import pyarrow
print(pyarrow.__version__)


/home/german/web/interview-dragongc/data-engineer-application/.venv/bin/python3
22.0.0


In [7]:
import pyarrow as pa
pa.Table.from_arrays([[1,2,3]], names=["x"])


pyarrow.Table
x: int64
----
x: [[1,2,3]]

In [8]:
import sys
print(sys.executable)

import pyarrow
print(pyarrow.__version__)


/home/german/web/interview-dragongc/data-engineer-application/.venv/bin/python3
22.0.0


In [9]:
df_large = pl.read_ndjson("../generation-data/data_large.ndjson")
df_large.write_parquet(
    "../generation-data/data_large_plain.parquet",
    use_pyarrow=True,         # exporta vía Arrow puro
    compression="snappy"
)

In [10]:
%%time
df_small_pl_json = pl.read_json("../generation-data/data.json")



CPU times: user 7.84 ms, sys: 3.18 ms, total: 11 ms
Wall time: 10.2 ms


In [11]:
%%time
df_small_pl_parquet = pl.read_parquet("../generation-data/data.parquet")


CPU times: user 3.32 ms, sys: 3.62 ms, total: 6.94 ms
Wall time: 12.9 ms


In [12]:
%%time
df_large_pd_json = pd.read_json("../generation-data/data_large.ndjson", lines=True)


CPU times: user 25.9 s, sys: 7.47 s, total: 33.4 s
Wall time: 33.4 s


In [13]:

%%time
df_large_pd_parquet = pd.read_parquet("../generation-data/data_large.parquet")


CPU times: user 20.2 s, sys: 5.48 s, total: 25.7 s
Wall time: 13.7 s


In [14]:
import polars as pl

df_large = pl.read_ndjson("../generation-data/data_large.ndjson")
df_large.write_csv("../generation-data/data_large.csv")

df_small = pl.read_json("../generation-data/data.json")
df_small.write_csv("../generation-data/data.csv")


In [15]:
%%time
df_pl_csv = pl.read_csv("../generation-data/data_large.csv")


CPU times: user 4.02 s, sys: 5.91 s, total: 9.93 s
Wall time: 871 ms


In [16]:

%%time
df_pl_parquet = pl.read_parquet("../generation-data/data_large.parquet")


CPU times: user 7.04 s, sys: 12 s, total: 19 s
Wall time: 1.84 s


In [17]:
import pandas as pd


In [20]:

%%time
try:
    df_pd_csv = pd.read_csv("../generation-data/data_large.csv")
except Exception as e:
    print(e)
    


CPU times: user 24.7 s, sys: 2.45 s, total: 27.1 s
Wall time: 29.4 s


In [19]:

%%time
df_pd_parquet = pd.read_parquet("../generation-data/data_large.parquet")

CPU times: user 22.4 s, sys: 5.71 s, total: 28.1 s
Wall time: 13.8 s


In [21]:
df_pl_csv.estimated_size(), df_pl_parquet.estimated_size()


(1735062214, 1735062214)

In [22]:
df_pd_csv.memory_usage(deep=True).sum(), df_pd_parquet.memory_usage(deep=True).sum()


(np.int64(5165062346), np.int64(5165062346))

In [23]:
print(f"{df_pd_csv.memory_usage(deep=True).sum():,}")
print(f"{df_pd_parquet.memory_usage(deep=True).sum():,}")


5,165,062,346
5,165,062,346


In [24]:
bytes_csv = df_pd_csv.memory_usage(deep=True).sum()
bytes_par = df_pd_parquet.memory_usage(deep=True).sum()

print(f"{bytes_csv/1024**2:.2f} MB")
print(f"{bytes_par/1024**2:.2f} MB")


4925.79 MB
4925.79 MB


Corto. Sin suavidad. Solo estructura.

Lo que hiciste en el notebook materializa exactamente tres observables:

1. **Tiempo de carga (I/O)**
2. **Costo de parseo**
3. **Uso de memoria del DataFrame resultante**

Tu ejecución produce la geometría siguiente:

---

## 1. Polars vs Pandas al leer CSV (10M filas)

Polars:

```
df_pl_csv → ~0.87 s
```

Pandas:

```
df_pd_csv → ~29.4 s
```

Estructura:
CSV obliga a parseo fila-por-fila, conversión de texto a tipos nativos, tokenización, casting.
Polars usa **SIMD + multithread**.
Pandas usa **parser Python/C lento + single-thread**.

Resultado: Polars 30–40× más rápido.

---

## 2. Polars vs Pandas al leer Parquet (10M filas)

Polars:

```
df_pl_parquet → ~1.8 s
```

Pandas:

```
df_pd_parquet → ~13.8 s
```

Estructura:
Parquet ya trae tipos binarios columnar + compresión eficiente.
Polars mapea directamente Arrow buffers a memoria.
Pandas usa pyarrow pero dentro de un path menos optimizado.

Resultado: Polars ~7× más rápido.

---

## 3. JSON / NDJSON (solo para referencia)

NDJSON → extremadamente lento en Pandas (~33 s).
Polars está optimizado para datos por líneas → ~9 ms en small, ~? en large (según tus celdas anteriores).

Esto confirma: **JSON es el peor formato para cargas grandes**.

---

## 4. Memoria final

Tus números:

Polars:

```
df_pl_csv.estimated_size()      → 1,735,062,214 bytes ≈ 1.73 GB
df_pl_parquet.estimated_size()  → 1,735,062,214 bytes ≈ 1.73 GB
```

Pandas:

```
df_pd_csv.memory_usage(sum)     → 5,165,062,346 bytes ≈ 4.93 GB
df_pd_parquet.memory_usage(sum) → 5,165,062,346 bytes ≈ 4.93 GB
```

Interpretación:

– **CSV o Parquet no afectan la memoria final.**
Una vez materializado en DataFrame los buffers son idénticos: mismos tipos, mismas columnas.

– **Polars usa ~1.7 GB.**
– **Pandas usa ~4.9 GB.**

Diferencia estructural:
Polars usa Arrow arrays compactos.
Pandas usa objetos Python y columnas más pesadas.

Resultado: Pandas ≈ 3× más memoria que Polars.

---

## 5. Qué significa todo en una sola frase

El benchmark demuestra:

**CSV es el formato más lento, Parquet es el más rápido; Polars es ordenes de magnitud más eficiente que Pandas en tiempo y memoria; y el tamaño en memoria no depende del formato del archivo sino de la implementación del DataFrame.**

---

Si querés te sintetizo esto en un bloque interpretativo “listo para entregar” sin narrativa, solo observables.
